[View in Colaboratory](https://colab.research.google.com/github/ozanpekmezci/capstone/blob/master/Digit_Recognition.ipynb)

# Recognition of Digits from Images
This notebook serves as the first step of the project that recognizes house numbers from live camera image. In this first step, MNIST database is used.

In [1]:

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


11501568/11490434 [==============================] - 1s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 13s 221us/step - loss: 0.2676 - acc: 0.9164 - val_loss: 0.0576 - val_acc: 0.9814
Epoch 2/12
17152/60000 [=======>......................] - ETA: 7s - loss: 0.0987 - acc: 0.9707

60000/60000 [==============================] - 11s 190us/step - loss: 0.0888 - acc: 0.9740 - val_loss: 0.0409 - val_acc: 0.9860
Epoch 3/12
60000/60000 [==============================] - 11s 190us/step - loss: 0.0665 - acc: 0.9808 - val_loss: 0.0329 - val_acc: 0.9898
Epoch 4/12
54400/60000 [==========================>...] - ETA: 1s - loss: 0.0545 - acc: 0.9840

60000/60000 [==============================] - 11s 189us/step - loss: 0.0535 - acc: 0.9841 - val_loss: 0.0299 - val_acc: 0.9906
Epoch 5/12
60000/60000 [==============================] - 12s 193us/step - loss: 0.0465 - acc: 0.9862 - val_loss: 0.0275 - val_acc: 0.9909
Epoch 6/12
60000/60000 [==============================] - 12s 194us/step - loss: 0.0398 - acc: 0.9878 - val_loss: 0.0274 - val_acc: 0.9910


Epoch 7/12
60000/60000 [==============================] - 11s 191us/step - loss: 0.0389 - acc: 0.9879 - val_loss: 0.0258 - val_acc: 0.9910
Epoch 8/12
60000/60000 [==============================] - 12s 192us/step - loss: 0.0343 - acc: 0.9893 - val_loss: 0.0241 - val_acc: 0.9926
Epoch 9/12
44160/60000 [=====================>........] - ETA: 2s - loss: 0.0309 - acc: 0.9906

60000/60000 [==============================] - 12s 192us/step - loss: 0.0310 - acc: 0.9906 - val_loss: 0.0264 - val_acc: 0.9911
Epoch 10/12
60000/60000 [==============================] - 12s 192us/step - loss: 0.0328 - acc: 0.9903 - val_loss: 0.0269 - val_acc: 0.9913
Epoch 11/12
58880/60000 [============================>.] - ETA: 0s - loss: 0.0282 - acc: 0.9913

60000/60000 [==============================] - 11s 191us/step - loss: 0.0282 - acc: 0.9913 - val_loss: 0.0264 - val_acc: 0.9912
Epoch 12/12
60000/60000 [==============================] - 11s 191us/step - loss: 0.0272 - acc: 0.9918 - val_loss: 0.0262 - val_acc: 0.9909
Test loss: 0.02615504841945658
Test accuracy: 0.9909


In [37]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

Saving image_1.jpg to image_1.jpg
User uploaded file "image_1.jpg" with length 3789530 bytes


In [41]:
import cv2
import numpy
img = cv2.imread('image_1.jpg')
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_gray = cv2.resize(img_gray, (img_cols, img_rows))
arr = numpy.array(img_gray).reshape((img_cols,img_rows,1))
arr = numpy.expand_dims(arr, axis=0)
prediction = model.predict(arr)
print(prediction)

[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
